# Description

This notebook is similar to `30` and `35`, but here I use the LVs that we found to be significantly enriched for the lipids CRISPR analysis, which might or might not coincide with the previously used LVs (those that discriminate clusters).
The traits here are from PhenomeXcan.

# Environment variables

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import conf

In [3]:
N_JOBS = conf.GENERAL["N_JOBS"]
display(N_JOBS)

3

In [4]:
%env MKL_NUM_THREADS=$N_JOBS
%env OPEN_BLAS_NUM_THREADS=$N_JOBS
%env NUMEXPR_NUM_THREADS=$N_JOBS
%env OMP_NUM_THREADS=$N_JOBS

env: MKL_NUM_THREADS=3
env: OPEN_BLAS_NUM_THREADS=3
env: NUMEXPR_NUM_THREADS=3
env: OMP_NUM_THREADS=3


# Modules

In [5]:
from pathlib import Path

import pandas as pd
from tqdm import tqdm

from gls import GLSPhenoplier

# Settings

In [6]:
N_TOP_TRAITS_FROM_LV = 20

In [7]:
OUTPUT_DIR = conf.RESULTS["GLS"]
display(OUTPUT_DIR)

OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/gls')

In [8]:
OUTPUT_FILENAME = OUTPUT_DIR / "gls_phenotypes-crispr_lvs-phenomexcan.pkl"
display(OUTPUT_FILENAME)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/gls/gls_phenotypes-crispr_lvs-phenomexcan.pkl')

# Load data

## PhenomeXcan (S-MultiXcan)

In [9]:
INPUT_SUBSET = "z_score_std"

In [10]:
INPUT_STEM = "projection-smultixcan-efo_partial-mashr-zscores"

In [11]:
input_filepath = Path(
    conf.RESULTS["DATA_TRANSFORMATIONS_DIR"],
    INPUT_SUBSET,
    f"{INPUT_SUBSET}-{INPUT_STEM}.pkl",
).resolve()

In [12]:
phenomexcan_projection = pd.read_pickle(input_filepath)

In [13]:
phenomexcan_projection.shape

(3752, 987)

In [14]:
phenomexcan_projection.head()

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
100001_raw-Food_weight,-0.695006,1.962565,0.057683,0.878731,-0.539977,1.481272,-0.396422,1.090180,0.759223,0.931395,...,1.129784,1.752343,-1.411403,2.823863,0.931116,-1.054519,0.432982,-0.633597,0.554279,-0.642479
100002_raw-Energy,-1.528127,-0.345309,-0.148953,-0.242060,0.373427,0.791092,0.263477,0.987702,0.354391,1.416059,...,0.224604,0.769882,-0.509482,0.091153,2.286789,-1.008256,-0.029764,1.737229,-0.272107,-0.526125
100003_raw-Protein,-0.704572,-1.011299,0.671420,0.143991,0.615212,0.874212,-0.040998,0.915170,0.254369,-0.084237,...,1.003019,1.044314,-2.376108,0.004778,0.053714,-0.892447,-0.183800,1.377991,-0.278794,-0.419733
100004_raw-Fat,-0.989832,-1.875490,0.261555,-1.420719,0.366238,1.167049,0.257387,0.717674,-0.997664,0.969825,...,0.585913,0.638314,0.119139,-0.140204,1.394326,-1.173402,0.555058,1.013982,-0.544506,-0.064061
100005_raw-Carbohydrate,-0.580143,0.243335,0.158966,-0.036558,0.068176,-0.202639,1.101281,0.675227,1.463432,1.010078,...,-0.249108,-0.026814,0.232713,0.323682,1.168642,-0.282935,0.653105,1.909526,0.199997,-1.656894


## Clustering results

In [15]:
CONSENSUS_CLUSTERING_DIR = Path(
    conf.RESULTS["CLUSTERING_DIR"], "consensus_clustering"
).resolve()

display(CONSENSUS_CLUSTERING_DIR)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/clustering/consensus_clustering')

In [16]:
input_file = Path(CONSENSUS_CLUSTERING_DIR, "best_partitions_by_k.pkl").resolve()
display(input_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/clustering/consensus_clustering/best_partitions_by_k.pkl')

In [17]:
best_partitions = pd.read_pickle(input_file)

In [18]:
best_partitions.shape

(59, 4)

In [19]:
best_partitions.head()

,method,partition,ari_median,selected
k,,,,
14,scc_025,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.090117,True
22,scc_020,"[13, 18, 18, 18, 18, 18, 18, 18, 18, 13, 18, 1...",0.090100,True
13,scc_025,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.089920,True
12,scc_025,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.089894,True
11,scc_025,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.089616,True


## MultiPLIER summary

In [20]:
multiplier_model_summary = pd.read_pickle(conf.MULTIPLIER["MODEL_SUMMARY_FILE"])

In [21]:
multiplier_model_summary.shape

(2157, 5)

In [22]:
multiplier_model_summary.head()

,pathway,LV index,AUC,p-value,FDR
1,KEGG_LYSINE_DEGRADATION,1,0.388059,0.866078,0.956005
2,REACTOME_MRNA_SPLICING,1,0.733057,0.000048,0.000582
3,MIPS_NOP56P_ASSOCIATED_PRE_RRNA_COMPLEX,1,0.680555,0.001628,0.011366
4,KEGG_DNA_REPLICATION,1,0.549473,0.312155,0.539951
5,PID_MYC_ACTIVPATHWAY,1,0.639303,0.021702,0.083739


In [23]:
well_aligned_lvs = multiplier_model_summary[
    (multiplier_model_summary["FDR"] < 0.05) | (multiplier_model_summary["AUC"] >= 0.75)
]

display(well_aligned_lvs.shape)
display(well_aligned_lvs.head())

(469, 5)

,pathway,LV index,AUC,p-value,FDR
2,REACTOME_MRNA_SPLICING,1,0.733057,4.772691e-05,5.816211e-04
3,MIPS_NOP56P_ASSOCIATED_PRE_RRNA_COMPLEX,1,0.680555,1.628217e-03,1.136590e-02
8,REACTOME_MITOTIC_G1_G1_S_PHASES,1,0.686170,2.517619e-04,2.392292e-03
9,IRIS_Monocyte-Day0,2,0.890036,4.315812e-25,1.329887e-22
10,DMAP_MONO2,2,0.904676,1.313970e-16,1.574574e-14


In [24]:
well_aligned_lv_codes = set([f"LV{lvi}" for lvi in well_aligned_lvs["LV index"]])

In [25]:
len(well_aligned_lv_codes)

200

In [26]:
list(well_aligned_lv_codes)[:5]

['LV675', 'LV49', 'LV176', 'LV913', 'LV599']

# Select LVs from CRISPR analysis

In [27]:
# FIXME: there will be a specific folder for crispr analysis in the future, that should be replaced here
deg_enrich = pd.read_csv(
    Path(
        conf.RESULTS["BASE_DIR"],
        "crispr_analyses",
        "fgsea-hi_conf-all_lvs.tsv",
    ).resolve(),
    sep="\t",
)

In [28]:
deg_enrich.shape

(1974, 11)

In [29]:
deg_enrich.head()

,pathway,pval,padj,log2err,ES,NES,size,leadingEdge,lv,rep_idx,fdr
0,gene_set_decrease,0.116883,0.233766,0.125640,0.909509,1.237006,5,"PCYT2, UBE2J2, FBXW7",LV1,8,0.967285
1,gene_set_increase,0.285714,0.285714,0.072180,0.840253,1.258282,3,"ACACA, MBTPS1",LV1,2,0.967285
2,gene_set_decrease,0.741259,0.741259,0.026956,0.751964,1.005845,5,"TCF7L2, UBE2J2, PTEN",LV10,10,0.967285
3,gene_set_increase,0.427572,0.720280,0.052805,0.814140,1.320263,3,"MBTPS1, DGAT2",LV10,4,0.967285
4,gene_set_decrease,0.908092,0.908092,0.014514,0.540400,0.778719,5,"PTEN, TCF7L2",LV100,1,0.974250


In [30]:
# deg_enrich_max_idx = deg_enrich.groupby(["lv", "pathway"])["pval"].idxmax()

In [31]:
# deg_enrich = deg_enrich.loc[deg_enrich_max_idx].reset_index(drop=True)
# display(deg_enrich.shape)
# display(deg_enrich.head())

## Lipids-increasing gene sets

In [32]:
deg_increase = deg_enrich[
    deg_enrich["pathway"].isin(("gene_set_increase",)) & (deg_enrich["pval"] < 0.01)
].sort_values("pval", ascending=True)

In [33]:
deg_increase.shape

(7, 11)

In [34]:
deg_increase.head()

,pathway,pval,padj,log2err,ES,NES,size,leadingEdge,lv,rep_idx,fdr
327,gene_set_increase,0.003533,0.007067,0.431708,0.998221,1.582398,3,"DGAT2, ACACA",LV246,6,0.967285
1341,gene_set_increase,0.004573,0.009147,0.407018,0.998666,1.495780,3,"ACACA, DGAT2",LV702,9,0.967285
1129,gene_set_increase,0.005837,0.011675,0.407018,0.996739,1.510660,3,"ACACA, DGAT2",LV607,10,0.967285
1757,gene_set_increase,0.006730,0.013459,0.407018,0.997480,1.545604,3,"ACACA, DGAT2",LV890,10,0.967285
1423,gene_set_increase,0.007847,0.015693,0.380730,0.990514,1.474472,3,"MBTPS1, DGAT2",LV74,6,0.967285


In [35]:
lvs_increase = deg_increase["lv"].unique()

In [36]:
lvs_increase.shape

(7,)

In [37]:
lvs_increase

array(['LV246', 'LV702', 'LV607', 'LV890', 'LV74', 'LV865', 'LV841'],
      dtype=object)

## Lipids-decreasing gene sets

In [38]:
deg_decrease = deg_enrich[
    deg_enrich["pathway"].isin(("gene_set_decrease",)) & (deg_enrich["pval"] < 0.01)
].sort_values("pval", ascending=True)

In [39]:
deg_decrease.shape

(8, 11)

In [40]:
deg_decrease.head()

,pathway,pval,padj,log2err,ES,NES,size,leadingEdge,lv,rep_idx,fdr
936,gene_set_decrease,0.000554,0.001108,0.477271,0.999703,1.388986,5,"FBXW7, TCF7L2",LV520,6,0.967285
1560,gene_set_decrease,0.002201,0.004403,0.431708,0.997776,1.359600,5,"UBE2J2, TCF7L2",LV801,6,0.967285
918,gene_set_decrease,0.002460,0.004921,0.431708,0.997331,1.382906,5,"FBXW7, TCF7L2",LV512,2,0.967285
1140,gene_set_decrease,0.003570,0.007141,0.431708,0.999299,1.400503,5,"PTEN, FBXW7",LV612,3,0.967285
690,gene_set_decrease,0.004053,0.008105,0.407018,0.996738,1.348694,5,"PCYT2, TCF7L2",LV41,4,0.967285


In [41]:
lvs_decrease = deg_decrease["lv"].unique()

In [42]:
lvs_decrease.shape

(8,)

In [43]:
lvs_decrease

array(['LV520', 'LV801', 'LV512', 'LV612', 'LV41', 'LV838', 'LV302',
       'LV959'], dtype=object)

## Merge into one dataframe

In [44]:
_tmp0 = pd.DataFrame({"lv": lvs_increase, "lv_set": "lipids-increasing"})

_tmp1 = pd.DataFrame({"lv": lvs_decrease, "lv_set": "lipids-decreasing"})

In [45]:
gls_selected_lvs = pd.concat([_tmp0, _tmp1], ignore_index=True)

In [46]:
gls_selected_lvs.shape

(15, 2)

In [47]:
gls_selected_lvs.head()

,lv,lv_set
0,LV246,lipids-increasing
1,LV702,lipids-increasing
2,LV607,lipids-increasing
3,LV890,lipids-increasing
4,LV74,lipids-increasing


# Select traits from specific partition/cluster

For this run on the LVs related to the lipids CRISPR analysis, I'm only interested in the main clusters of the cardiovascular sub-branch.

In [48]:
# PHENOTYPES_CONFIG = [
#     # cardiovascular
#     (29, 14),
#     (29, 16),
#     (29, 11),
#     (29, 21),
#     (29, 17),
# ]

# GLSPhenoplier

## Get list of phenotypes/lvs pairs

In [49]:
phenotypes_lvs_pairs = []

# for each LV, I take the top `N_TOP_TRAITS_FROM_LV` traits in eMERGE
for idx, row in gls_selected_lvs.iterrows():
    lv_name = row["lv"]
    lv_set = row["lv_set"]

    lv_traits = phenomexcan_projection[lv_name]
    lv_traits = lv_traits[lv_traits > 0.0]
    lv_traits = lv_traits.sort_values(ascending=False).head(N_TOP_TRAITS_FROM_LV)

    for phenotype_code in set(lv_traits.index.tolist()):
        phenotypes_lvs_pairs.append(
            {
                "phenotype": phenotype_code,
                "lv": lv_name,
                "lv_set": row["lv_set"],
            }
        )

phenotypes_lvs_pairs = pd.DataFrame(phenotypes_lvs_pairs).drop_duplicates()

In [50]:
phenotypes_lvs_pairs = phenotypes_lvs_pairs.sort_values("phenotype").reset_index(
    drop=True
)

In [51]:
phenotypes_lvs_pairs.shape

(300, 3)

In [52]:
phenotypes_lvs_pairs.head()

,phenotype,lv,lv_set
0,100007_raw-Polyunsaturated_fat,LV890,lipids-increasing
1,100360-Decaffeinated_coffee,LV520,lipids-decreasing
2,102280-Milk_chocolate_intake,LV702,lipids-increasing
3,102_raw-Pulse_rate_automated_reading,LV246,lipids-increasing
4,102_raw-Pulse_rate_automated_reading,LV801,lipids-decreasing


## Run

In [53]:
results = []

pbar = tqdm(total=phenotypes_lvs_pairs.shape[0])

for idx, row in phenotypes_lvs_pairs.iterrows():
    phenotype_code = row["phenotype"]
    lv_code = row["lv"]

    pbar.set_description(f"{phenotype_code} - {lv_code}")

    gls_model = GLSPhenoplier(
        smultixcan_result_set_filepath=conf.PHENOMEXCAN[
            "SMULTIXCAN_EFO_PARTIAL_MASHR_ZSCORES_FILE"
        ]
    )
    gls_model.fit_named(lv_code, phenotype_code)
    res = gls_model.results

    results.append(
        {
            "phenotype": phenotype_code,
            "lv": lv_code,
            "lv_set": row["lv_set"],
            "lv_with_pathway": lv_code in well_aligned_lv_codes,
            "coef": res.params.loc["lv"],
            "pvalue": res.pvalues_onesided.loc["lv"],
            "pvalue_twosided": res.pvalues.loc["lv"],
            "summary": gls_model.results_summary,
        }
    )

    # save results every 10 models trained
    if (idx % 10) == 0:
        pd.DataFrame(results).to_pickle(OUTPUT_FILENAME)

    pbar.update(1)

pbar.close()

renal colic - LV865: 100%|██████████| 300/300 [47:11<00:00,  9.44s/it]


In [54]:
results = pd.DataFrame(results)

In [55]:
results.shape

(300, 8)

In [56]:
results.head()

,phenotype,lv,lv_set,lv_with_pathway,coef,pvalue,pvalue_twosided,summary
0,100007_raw-Polyunsaturated_fat,LV890,lipids-increasing,False,0.044519,0.000228,0.000455,GLS Regression Res...
1,100360-Decaffeinated_coffee,LV520,lipids-decreasing,False,0.041107,0.000391,0.000782,GLS Regression Res...
2,102280-Milk_chocolate_intake,LV702,lipids-increasing,False,0.043164,0.000335,0.000670,GLS Regression Res...
3,102_raw-Pulse_rate_automated_reading,LV246,lipids-increasing,True,0.037216,0.001377,0.002754,GLS Regression Res...
4,102_raw-Pulse_rate_automated_reading,LV801,lipids-decreasing,False,0.048699,0.000066,0.000131,GLS Regression Res...


In [57]:
results.sort_values("pvalue").head(10)

,phenotype,lv,lv_set,lv_with_pathway,coef,pvalue,pvalue_twosided,summary
156,23108_raw-Impedance_of_leg_left,LV841,lipids-increasing,False,0.111498,3.384759e-19,6.769517e-19,GLS Regression Res...
207,50_raw-Standing_height,LV841,lipids-increasing,False,0.109778,1.940273e-18,3.880546e-18,GLS Regression Res...
63,20015_raw-Sitting_height,LV841,lipids-increasing,False,0.104691,9.201681e-17,1.840336e-16,GLS Regression Res...
153,23101_raw-Whole_body_fatfree_mass,LV841,lipids-increasing,False,0.104054,1.422409e-16,2.844818e-16,GLS Regression Res...
157,23113_raw-Leg_fatfree_mass_right,LV841,lipids-increasing,False,0.104337,1.747263e-16,3.494525e-16,GLS Regression Res...
159,23117_raw-Leg_fatfree_mass_left,LV841,lipids-increasing,False,0.104339,1.783712e-16,3.567423e-16,GLS Regression Res...
164,23129_raw-Trunk_fatfree_mass,LV841,lipids-increasing,False,0.103358,1.839193e-16,3.678386e-16,GLS Regression Res...
165,23130_raw-Trunk_predicted_mass,LV841,lipids-increasing,False,0.103278,1.914489e-16,3.828978e-16,GLS Regression Res...
158,23114_raw-Leg_predicted_mass_right,LV841,lipids-increasing,False,0.104124,2.005663e-16,4.011327e-16,GLS Regression Res...
160,23118_raw-Leg_predicted_mass_left,LV841,lipids-increasing,False,0.104156,2.016841e-16,4.033682e-16,GLS Regression Res...


## Save

In [58]:
results.to_pickle(OUTPUT_FILENAME)